In [1]:
from __future__ import division
import numpy as np
import glob
import csv

def readcsv(filename):
    with open(filename, 'r') as dest_f:
        data_iter = csv.reader(dest_f, delimiter=',', quotechar='"')
        data = [data for data in data_iter]
    data_array = np.asarray(data)
    return data_array


def removeRows(var, data, a):
    del_len = int(np.shape(var)[0])
    del_rows = []
    for i in range(0, a):
        for k in range(0, del_len):
            if(data[i, 0]==var[k]):
                del_rows.append(i)
    data = np.delete(data, del_rows, axis=0)
                      
    return data

In [2]:
def cvsTOndarray(dataTag, fld, outputFLD,extra_col):
    for year in range(0, 3):
        file_base_burn = fld[228*(year)]
        data_base_burn = readcsv(file_base_burn)
        shape_base_burn = np.shape(data_base_burn)
        a = shape_base_burn[0] - 1
        b = shape_base_burn[1] 
        data_all = np.zeros((waterYearLen, 12, a, b+extra_col))
        
        
        fld_unbn = sorted(glob.glob(path +'Unbn'+ '_0909_100km/*.csv'))
        file_base_unbn = fld_unbn[228*(year)]
        data_base_unbn = readcsv(file_base_unbn)
        
        ID_burn = data_base_burn[1:a+1, 0]
        ID_unbn = data_base_unbn[1:a+1, 0]
        
        ID_dif_burn = list(set(ID_burn) - set(ID_unbn))
        ID_dif_unbn = list(set(ID_unbn) - set(ID_burn))
        ID_dif=np.append(ID_dif_burn, ID_dif_unbn)
       
        
        if dataTag == 'Burn':
            data_all = np.zeros((19, 12, a-np.shape(ID_dif_burn)[0], b+extra_col))
            var = np.array(ID_dif_burn).astype(float)
        if dataTag == 'Unbn':
            data_all = np.zeros((19, 12, a-np.shape(ID_dif_unbn)[0], b+extra_col)) 
            var = np.array(ID_dif_unbn).astype(float)
   
        for i in range(months_total*(year), months_total*(year+1)):
            fireYear = waterYearStart+year
            file = fld[i]
#             print(file)
            data = readcsv(file)
            data = data[1:a+1,0:b]
            data[data == ''] = np.nan
            data = data.astype(float)
            data = sorted(data, key=lambda x: x[0])
            data = np.array(data)  
            
            data = removeRows(var, data, a)
            mask_length = a- np.shape(var)[0]
          
            
            waterYear = np.floor((i-228*(year))/12)
            waterYear = int(waterYear)
            month = int(i%12)

            data_all[waterYear, month, :, 0:b] = data
            data_all[waterYear, month, :, 11] = 10*data_all[waterYear, month, :, 11]
            data_all[waterYear, month, :, 21] = 10*data_all[waterYear, month, :, 21]
            data_all[waterYear, month, :, 12] = 100*data_all[waterYear, month, :, 12]
            
            data_all[waterYear, month, :, b] = data[:,20]-data[:,19]
            data_all[waterYear, month, :, b+1] = data[:,8] - data[:,7] 
            data_all[waterYear, month, :, b+2] = data[:,11] - data[:,7]
            data_all[waterYear, month, :, b+3] = data[:,11] - data[:,8]
            
            data_all[waterYear, month, :, b+4] = data[:,9]*1.1*10
            mask = np.zeros((mask_length,1))
            index_1 = np.where(data[:,9]<0)
            mask[index_1,0] = 0
            index_2 = np.logical_and(data[:,10]<1,data[:,9]>0)
            
            mask[index_2,0] = data[index_2,10]*0.0133+0.0667
            index_3 = np.where(data[:,9]>=1)
            mask[index_3,0] = 0.08
            data_all[waterYear, month, :, b+4] =data_all[waterYear, month, :, b+4]*mask[:,0]
            mask_rad = np.zeros((a,1))
            index_rad = np.where(data[:,17]>0)
            mask[index_rad,0] = 1
            data_all[waterYear, month, :, b+4] = data_all[waterYear, month, :, b+4] + 0.026*data[:,17]*mask[:,0]
            
            
            data_all[waterYear, month, :, b+5] = data[:,10]*1.1*10
            mask = np.zeros((mask_length,1))
            index_1 = np.where(data[:,10]<0)
            mask[index_1,0] = 0
            index_2 = np.logical_and(data[:,10]<1,data[:,10]>0)
            mask[index_2,0] = data[index_2,10]*0.0133+0.0667
            index_3 = np.where(data[:,10]>=1)
            mask[index_3,0] = 0.08
            data_all[waterYear, month, :, b+5] =data_all[waterYear, month, :, b+5]*mask[:,0]
            mask_rad = np.zeros((a,1))
            index_rad = np.where(data[:,16]>0)
            mask[index_rad,0] = 1
            data_all[waterYear, month, :, b+5] = data_all[waterYear, month, :, b+5] + 0.026*data[:,16]*mask[:,0]
            
            
            mask_temp = np.zeros((mask_length,1))
            index_temp = np.where(data[:,22]>0)
            mask_temp[index_temp,0] = 1
            mask_rad = np.zeros((mask_length,1))
            index_rad = np.where(data[:,23]>0)
            mask_rad[index_rad,0] = 1
            data_all[waterYear, month, :, b+6] = data[:,22]*mask_temp[:,0] + data[:,23]*mask_rad[:,0]
            mask_temp = np.zeros((mask_length,1))
            index_temp = np.where(data[:,25]>0)
            mask_temp[index_temp,0] = 1
            mask_rad = np.zeros((mask_length,1))
            index_rad = np.where(data[:,26]>0)
            mask_rad[index_rad,0] = 1
            data_all[waterYear, month, :, b+7] = data[:,25]*mask_temp[:,0] + data[:,26]*mask_rad[:,0]
            mask_temp = np.zeros((mask_length,1))
            index_temp = np.where(data[:,22]>0)
            mask_temp[index_temp,0] = 1
            mask_rad = np.zeros((mask_length,1))
            index_rad = np.where(data[:,29]>0)
            mask_rad[index_rad,0] = 1
            data_all[waterYear, month, :, b+8] = data[:,22]*mask_temp[:,0] + data[:,29]*mask_rad[:,0]
            mask_temp = np.zeros((mask_length,1))
            index_temp = np.where(data[:,25]>0)
            mask_temp[index_temp,0] = 1
            mask_rad = np.zeros((mask_length,1))
            index_rad = np.where(data[:,27]>0)
            mask_rad[index_rad,0] = 1
            data_all[waterYear, month, :, b+9] = data[:,25]*mask_temp[:,0] + data[:,27]*mask_rad[:,0]

            data_all[waterYear, month, :, b+10] = data[:,21] 
        
        for j in range(0, waterYearLen): 
            for k in range(0, 1):
                for l in range(0, mask_length):
                    for index in [-20,4, 5,6,7,8,9]:
                        if data_all[j,k,l,b+index]>data_all[j,k,l,b+10]:
                            data_all[j,k,l,b+index] = data_all[j,k,l,b+10]
                            data_all[j,k,l,b+10] = 0
                        else:
                            data_all[j,k,l,b+10] -= data_all[j,k,l,b+index]
                        
        
        for j in range(0, waterYearLen): 
            for k in range(1, 12):
                for l in range(0, mask_length):
                    for index in [-20,4, 5,6,7,8,9]:
                        data_all[j,k,l,b+10] += data_all[j,k-1,l,b+10]
                        
                        if data_all[j,k,l,b+index]>data_all[j,k,l,b+10]:
                            data_all[j,k,l,b+index] = data_all[j,k,l,b+10]
                            data_all[j,k,l,b+10] = 0
                        else:
                            data_all[j,k,l,b+10] -= data_all[j,k,l,b+index]
                        
                              
                    
        output = outputFLD + str(year+2003)
        np.save(output, data_all)

# Merge Data
extra_col = 11
waterYearLen = 19
months_total = 228
waterYearStart = 2002
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

dataTag = 'Burn'
fld = sorted(glob.glob(path +dataTag+ '_0909_100km/*.csv'))

outputFLD = path + dataTag + '_fireYear_'
cvsTOndarray(dataTag, fld, outputFLD,extra_col)

dataTag = 'Unbn'
fld = sorted(glob.glob(path +dataTag+ '_0909_100km/*.csv'))

outputFLD = path +dataTag+ '_fireYear_'
cvsTOndarray(dataTag, fld, outputFLD,extra_col)


In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Burn_2004 = np.load(path+'Burn_fireYear_2003.npy')
Data_Burn_2005 = np.load(path+'Burn_fireYear_2004.npy')
Data_Burn_2006 = np.load(path+'Burn_fireYear_2005.npy')

Data_Unbn_2004 = np.load(path+'Unbn_fireYear_2003.npy')
Data_Unbn_2005 = np.load(path+'Unbn_fireYear_2004.npy')
Data_Unbn_2006 = np.load(path+'Unbn_fireYear_2005.npy')

Data_Dif_2004 = Data_Burn_2004 - Data_Unbn_2004
Data_Dif_2005 = Data_Burn_2005 - Data_Unbn_2005
Data_Dif_2006 = Data_Burn_2006 - Data_Unbn_2006

Data_Dif_2004[:,:,:,0] = Data_Burn_2004[:,:,:,0]
Data_Dif_2004[:,:,:,1] = Data_Burn_2004[:,:,:,1]
Data_Dif_2004[:,:,:,2] = Data_Burn_2004[:,:,:,2]
Data_Dif_2004[:,:,:,30] = Data_Burn_2004[:,:,:,30]
Data_Dif_2004[:,:,:,31] = Data_Burn_2004[:,:,:,31]

Data_Dif_2005[:,:,:,0] = Data_Burn_2005[:,:,:,0]
Data_Dif_2005[:,:,:,1] = Data_Burn_2005[:,:,:,1]
Data_Dif_2005[:,:,:,2] = Data_Burn_2005[:,:,:,2]
Data_Dif_2005[:,:,:,30] = Data_Burn_2005[:,:,:,30]
Data_Dif_2005[:,:,:,31] = Data_Burn_2005[:,:,:,31]

Data_Dif_2006[:,:,:,0] = Data_Burn_2006[:,:,:,0]
Data_Dif_2006[:,:,:,1] = Data_Burn_2006[:,:,:,1]
Data_Dif_2006[:,:,:,2] = Data_Burn_2006[:,:,:,2]
Data_Dif_2006[:,:,:,30] = Data_Burn_2006[:,:,:,30]
Data_Dif_2006[:,:,:,31] = Data_Burn_2006[:,:,:,31]


print(np.shape(Data_Dif_2004))
print(np.shape(Data_Dif_2005))
print(np.shape(Data_Dif_2006))

output = path + 'Dif_fireYear_2003' 
np.save(output, Data_Dif_2004)

output = path+ 'Dif_fireYear_2004' 
np.save(output, Data_Dif_2005)

output = path + 'Dif_fireYear_2005' 
np.save(output, Data_Dif_2006)

(19, 12, 4990, 43)
(19, 12, 4664, 43)
(19, 12, 4805, 43)


In [17]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Dif_2004 = np.load(path + 'Dif_fireYear_2003.npy' )
Data_Dif_2005 = np.load(path+ 'Dif_fireYear_2004.npy' )
Data_Dif_2006 = np.load(path + 'Dif_fireYear_2005.npy' )
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)
size_2006 = np.shape(Data_Dif_2006)


len_points = size_2004[2]+size_2005[2]+size_2006[2]
len_var = size_2004[3]
Data_year = np.zeros((len_YearAF,len_points,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF):
    Data_year[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, :,:,:], axis=0)
    Data_year[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, :,:,:], axis=0)
    Data_year[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2, :,:,:], axis=0)
    
    
    Data_year_winter[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 0:6,:,:], axis=0)
    Data_year_winter[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 0:6,:,:], axis=0)
    Data_year_winter[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,0:6,:,:], axis=0)
    
    Data_year_summer[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 6:12,:,:], axis=0)
    Data_year_summer[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 6:12,:,:], axis=0)
    Data_year_summer[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,6:12,:,:], axis=0)
    
    
output_year = path  + 'Dif_YearlyAverage_17years_1BF_15AF'
np.save(output_year, Data_year)    
    
output_winter = path + 'Dif_WinterAverage_17years_1BF_15AF'
np.save(output_winter, Data_year_winter) 
    
output_summer = path + 'Dif_SummerAverage_17years_1BF_15AF'
np.save(output_summer, Data_year_summer) 


<ipython-input-17-fd9de6bdde38>:28: RuntimeWarning: Mean of empty slice
  Data_year[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, :,:,:], axis=0)
<ipython-input-17-fd9de6bdde38>:29: RuntimeWarning: Mean of empty slice
  Data_year[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, :,:,:], axis=0)
<ipython-input-17-fd9de6bdde38>:30: RuntimeWarning: Mean of empty slice
  Data_year[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2, :,:,:], axis=0)
<ipython-input-17-fd9de6bdde38>:33: RuntimeWarning: Mean of empty slice
  Data_year_winter[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 0:6,:,:], axis=0)
<ipython-input-17-fd9de6bdde38>:34: RuntimeWarning: Mean of empty slice
  Data_year_winter[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 0:6,:,:], axis=0)
<ipython-input-17-fd9de6bdde38>:35: RuntimeWarning: Mean of empty slice
  Data_year_winter[i,size_2004[2]+size_2005[2]:size_2004

In [15]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'
 
Data_Dif_2004 = np.load(path + 'Dif_fireYear_2003.npy' )
Data_Dif_2005 = np.load(path+ 'Dif_fireYear_2004.npy' )
Data_Dif_2006 = np.load(path + 'Dif_fireYear_2005.npy' )
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)
size_2006 = np.shape(Data_Dif_2006)
# len_YearAF = 15
len_points = size_2004[2]+size_2005[2]+size_2006[2]
len_var = size_2004[3]
Data_year = np.zeros((len_YearAF,len_points,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points,len_var))
Data_year_spring= np.zeros((len_YearAF,len_points,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points,len_var))
Data_year_fall = np.zeros((len_YearAF,len_points,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF):

    
    Data_year_winter[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 0:3,:,:], axis=0)
    Data_year_winter[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 0:3,:,:], axis=0)
    Data_year_winter[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,0:3,:,:], axis=0)
    
    Data_year_spring[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 3:6,:,:], axis=0)
    Data_year_spring[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 3:6,:,:], axis=0)
    Data_year_spring[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,3:6,:,:], axis=0)
    
    Data_year_summer[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 6:9,:,:], axis=0)
    Data_year_summer[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 6:9,:,:], axis=0)
    Data_year_summer[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,6:9,:,:], axis=0)
    
    Data_year_fall[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 9:12,:,:], axis=0)
    Data_year_fall[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 9:12,:,:], axis=0)
    Data_year_fall[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,9:12,:,:], axis=0)
    
    
  
    
output_winter = path + 'Dif_WinterAverage_17years_1BF_15AF_2'
np.save(output_winter, Data_year_winter) 
    
output_spring = path + 'Dif_SpringAverage_17years_1BF_15AF_2'
np.save(output_spring, Data_year_spring) 

    
output_summer = path + 'Dif_SummerAverage_17years_1BF_15AF_2'
np.save(output_summer, Data_year_summer) 
output_fall = path + 'Dif_FallAverage_17years_1BF_15AF_2'
np.save(output_fall, Data_year_fall)
import sys
sys.modules[__name__].__dict__.clear()

<ipython-input-15-079132596416>:31: RuntimeWarning: Mean of empty slice
  Data_year_winter[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 0:3,:,:], axis=0)
<ipython-input-15-079132596416>:32: RuntimeWarning: Mean of empty slice
  Data_year_winter[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 0:3,:,:], axis=0)
<ipython-input-15-079132596416>:33: RuntimeWarning: Mean of empty slice
  Data_year_winter[i,size_2004[2]+size_2005[2]:size_2004[2]+size_2005[2]+size_2006[2],:] = np.nanmean(Data_Dif_2006[i+2,0:3,:,:], axis=0)
<ipython-input-15-079132596416>:35: RuntimeWarning: Mean of empty slice
  Data_year_spring[i,0:size_2004[2],:] = np.nanmean(Data_Dif_2004[i, 3:6,:,:], axis=0)
<ipython-input-15-079132596416>:36: RuntimeWarning: Mean of empty slice
  Data_year_spring[i,size_2004[2]:size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+1, 3:6,:,:], axis=0)
<ipython-input-15-079132596416>:37: RuntimeWarning: Mean of empty slice
  Data_year_spring[i,size_2004

In [16]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Dif_2003 = np.load(path +'Burn_fireYear_2003.npy')
Data_Dif_2004 = np.load(path +'Burn_fireYear_2004.npy')
Data_Dif_2005 = np.load(path +'Burn_fireYear_2005.npy')
size_2003 = np.shape(Data_Dif_2003)
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)

Data_year = np.zeros((len_YearAF,len_points,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF):
    Data_year[i,0:size_2003[2],:] = np.nanmean(Data_Dif_2003[i, :,:,:], axis=0)
    Data_year[i,size_2003[2]:size_2003[2]+size_2004[2],:] = np.nanmean(Data_Dif_2004[i+1, :,:,:], axis=0)
    Data_year[i,size_2003[2]+size_2004[2]:size_2003[2]+size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+2, :,:,:], axis=0)
    
    
    Data_year_winter[i,0:size_2003[2],:] = np.nanmean(Data_Dif_2003[i, 0:6,:,:], axis=0)
    Data_year_winter[i,size_2003[2]:size_2003[2]+size_2004[2],:] = np.nanmean(Data_Dif_2004[i+1, 0:6,:,:], axis=0)
    Data_year_winter[i,size_2003[2]+size_2004[2]:size_2003[2]+size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+2,0:6,:,:], axis=0)
    
    Data_year_summer[i,0:size_2003[2],:] = np.nanmean(Data_Dif_2003[i, 6:12,:,:], axis=0)
    Data_year_summer[i,size_2003[2]:size_2003[2]+size_2004[2],:] = np.nanmean(Data_Dif_2004[i+1, 6:12,:,:], axis=0)
    Data_year_summer[i,size_2003[2]+size_2004[2]:size_2003[2]+size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+2, 6:12,:,:], axis=0)
    
    
output_year = path + 'Burn_YearlyAverage_17years_1BF_15AF'
np.save(output_year, Data_year)    
    
output_winter = path + 'Burn_WinterAverage_17years_1BF_15AF'
np.save(output_winter, Data_year_winter) 
    
output_summer = path + 'Burn_SummerAverage_17years_1BF_15AF'
np.save(output_summer, Data_year_summer) 
import sys
sys.modules[__name__].__dict__.clear()

NameError: name 'len_points' is not defined

In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Dif_2003 = np.load(path + 'Unbn_fireYear_2003.npy')
Data_Dif_2004 = np.load(path + 'Unbn_fireYear_2004.npy')
Data_Dif_2005 = np.load(path + 'Unbn_fireYear_2005.npy')
size_2003 = np.shape(Data_Dif_2003)
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)

Data_year = np.zeros((len_YearAF,len_points,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF):
    Data_year[i,0:size_2003[2],:] = np.nanmean(Data_Dif_2003[i, :,:,:], axis=0)
    Data_year[i,size_2003[2]:size_2003[2]+size_2004[2],:] = np.nanmean(Data_Dif_2004[i+1, :,:,:], axis=0)
    Data_year[i,size_2003[2]+size_2004[2]:size_2003[2]+size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+2, :,:,:], axis=0)
    
    
    Data_year_winter[i,0:size_2003[2],:] = np.nanmean(Data_Dif_2003[i, 0:6,:,:], axis=0)
    Data_year_winter[i,size_2003[2]:size_2003[2]+size_2004[2],:] = np.nanmean(Data_Dif_2004[i+1, 0:6,:,:], axis=0)
    Data_year_winter[i,size_2003[2]+size_2004[2]:size_2003[2]+size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+2,0:6,:,:], axis=0)
    
    Data_year_summer[i,0:size_2003[2],:] = np.nanmean(Data_Dif_2003[i, 6:12,:,:], axis=0)
    Data_year_summer[i,size_2003[2]:size_2003[2]+size_2004[2],:] = np.nanmean(Data_Dif_2004[i+1, 6:12,:,:], axis=0)
    Data_year_summer[i,size_2003[2]+size_2004[2]:size_2003[2]+size_2004[2]+size_2005[2],:] = np.nanmean(Data_Dif_2005[i+2, 6:12,:,:], axis=0)
    
    
output_year = path  + 'Unbn_YearlyAverage_17years_1BF_15AF'
np.save(output_year, Data_year)    
    
output_winter = path  + 'Unbn_WinterAverage_17years_1BF_15AF'
np.save(output_winter, Data_year_winter) 
    
output_summer = path  + 'Unbn_SummerAverage_17years_1BF_15AF'
np.save(output_summer, Data_year_summer) 
import sys
sys.modules[__name__].__dict__.clear()

In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Dif_2003 = np.load(path + 'Dif_fireYear_2003.npy')
Data_Dif_2004 = np.load(path + 'Dif_fireYear_2004.npy')
Data_Dif_2005 = np.load(path + 'Dif_fireYear_2005.npy')
size_2003 = np.shape(Data_Dif_2003)
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)

Data_year = np.zeros((len_YearAF,len_points*12,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points*6,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points*6,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF): 
    a=np.reshape(Data_Dif_2003[i,:,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,:,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,:,:,:],(-1,len_var))

    
    Data_year[i,:,:] = np.concatenate((a,b,c), axis=0)
    a=np.reshape(Data_Dif_2003[i,0:6,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,0:6,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,0:6,:,:],(-1,len_var))
    

    
    
    Data_year_winter[i,:,:] = np.concatenate((a,b,c), axis=0)
    a=np.reshape(Data_Dif_2003[i,6:12,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,6:12,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,6:12,:,:],(-1,len_var))
    

    
    Data_year_summer[i,:,:] = np.concatenate((a,b,c), axis=0)
                                           
    
    
output_year = path + 'Dif_YearlyAllData_17years_1BF_15AF'
np.save(output_year, Data_year)    
    
output_winter =path + 'Dif_WinterAllData_17years_1BF_15AF'
np.save(output_winter, Data_year_winter) 
    
output_summer = path + 'Dif_SummerAllData_17years_1BF_15AF'
np.save(output_summer, Data_year_summer) 
import sys
sys.modules[__name__].__dict__.clear()

In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Dif_2003 = np.load(path + 'Burn_fireYear_2003.npy')
Data_Dif_2004 = np.load(path + 'Burn_fireYear_2004.npy')
Data_Dif_2005 = np.load(path + 'Burn_fireYear_2005.npy')
size_2003 = np.shape(Data_Dif_2003)
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)

Data_year = np.zeros((len_YearAF,len_points*12,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points*6,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points*6,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF): 
    a=np.reshape(Data_Dif_2003[i,:,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,:,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,:,:,:],(-1,len_var))
    

    
    
    Data_year[i,:,:] = np.concatenate((a,b,c), axis=0)
    a=np.reshape(Data_Dif_2003[i,0:6,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,0:6,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,0:6,:,:],(-1,len_var))

    
    
    Data_year_winter[i,:,:] = np.concatenate((a,b,c), axis=0)
    a=np.reshape(Data_Dif_2003[i,6:12,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,6:12,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,6:12,:,:],(-1,len_var))
    

    
    Data_year_summer[i,:,:] = np.concatenate((a,b,c), axis=0)
                                           
    
    
output_year = path  + 'Burn_YearlyAllData_17years_1BF_15AF'
np.save(output_year, Data_year)    
    
output_winter = path + 'Burn_WinterAllData_17years_1BF_15AF'
np.save(output_winter, Data_year_winter) 
    
output_summer = path  + 'Burn_SummerAllData_17years_1BF_15AF'
np.save(output_summer, Data_year_summer) 
import sys
sys.modules[__name__].__dict__.clear()

In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

Data_Dif_2003 = np.load(path  + 'Unbn_fireYear_2003.npy')
Data_Dif_2004 = np.load(path  + 'Unbn_fireYear_2004.npy')
Data_Dif_2005 = np.load(path  + 'Unbn_fireYear_2005.npy')
size_2003 = np.shape(Data_Dif_2003)
size_2004 = np.shape(Data_Dif_2004)
size_2005 = np.shape(Data_Dif_2005)

Data_year = np.zeros((len_YearAF,len_points*12,len_var))
Data_year_winter = np.zeros((len_YearAF,len_points*6,len_var))
Data_year_summer = np.zeros((len_YearAF,len_points*6,len_var))
Data_year[:]=np.nan
Data_year_winter[:]=np.nan
Data_year_summer[:]=np.nan
for i in range(0, len_YearAF): 
    a=np.reshape(Data_Dif_2003[i,:,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,:,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,:,:,:],(-1,len_var))
    
 
    
    Data_year[i,:,:] = np.concatenate((a,b,c), axis=0)
    a=np.reshape(Data_Dif_2003[i,0:6,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,0:6,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,0:6,:,:],(-1,len_var))
    

    
    
    Data_year_winter[i,:,:] = np.concatenate((a,b,c), axis=0)
    a=np.reshape(Data_Dif_2003[i,6:12,:,:],(-1,len_var))
    b=np.reshape(Data_Dif_2004[i+1,6:12,:,:],(-1,len_var))
    c=np.reshape(Data_Dif_2005[i+2,6:12,:,:],(-1,len_var))

    
    
    Data_year_summer[i,:,:] = np.concatenate((a,b,c), axis=0)
                                           
                                           
    
    
output_year = path  + 'Unbn_YearlyAllData_17years_1BF_15AF'
np.save(output_year, Data_year)    
    
output_winter = path + 'Unbn_WinterAllData_17years_1BF_15AF'
np.save(output_winter, Data_year_winter) 
    
output_summer = path  + 'Unbn_SummerAllData_17years_1BF_15AF'
np.save(output_summer, Data_year_summer) 
import sys
sys.modules[__name__].__dict__.clear()

In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

def globalTObasin(dataTag):
    Data_Burn_2003 = np.load(path + dataTag + '_fireYear_2003.npy')
    Data_Burn_2004 = np.load(path + dataTag + '_fireYear_2004.npy')
    Data_Burn_2005 = np.load(path + dataTag + '_fireYear_2005.npy')
    basin_ID = [3020024310, 4020050210, 4020050220, 1020040190, 4020050290, 4020050470, 7020000010, 6020000010, 3020000010, 1020000010, 8020000010, 4020000010, 5020000010, 2020000010, 9020000010, 7020014250, 6020006540, 3020003790, 8020008900, 4020006940, 5020015660, 2020003440, 7020021430, 3020005240, 1020018110, 8020010700, 4020015090, 5020037270, 2020018240, 7020024600, 3020008670, 1020021940, 8020020760, 4020024190, 2020024230, 7020038340, 3020009320, 1020027430, 8020022890, 4020034510, 5020054880, 2020033490, 1020034170, 8020032840, 2020041390, 7020046750, 7020047840, 6020029280, 8020044560, 2020057170, 7020065090, 2020065840, 2020071190]
    data = np.zeros((waterYearLen, 12, 53, len_var))
    count_sum = 0
    data_all = np.zeros((3, waterYearLen, 12, 53, len_var))
    for year in range(0, 3):
        if year == 0:
            Data_fireYear = Data_Burn_2003
        if year == 1:
            Data_fireYear = Data_Burn_2004
        if year == 2:
            Data_fireYear = Data_Burn_2005
            

            
            
        for i in range(0, 53):
            count = np.count_nonzero(Data_fireYear[:, :, :, 2] == basin_ID[i])
            index = np.where(Data_fireYear[:,:,:,2] == basin_ID[i])
            index_x = index[0]
            index_y = index[1]
            index_z = index[2]
            rows = int(count / waterYearLen / 12)
            item_biomeType = np.zeros((waterYearLen, 12, rows, len_var))
            item_biomeType[:]=np.nan

            for j in range(0, count):
                item_biomeType[index_x[j], index_y[j], j % rows, :] = Data_fireYear[index_x[j], index_y[j], index_z[j], :]
            data[:,:,i,:] = np.nanmean(item_biomeType, axis=2)
            data_all[year, :, :, i, :] = np.nanmean(item_biomeType, axis=2)
    output = path + dataTag  + '_forEachBasin'
    np.save(output, data_all)

dataTag = 'Burn'
globalTObasin(dataTag)

dataTag = 'Unbn'
globalTObasin(dataTag)

dataTag = 'Dif'
globalTObasin(dataTag)
import sys
sys.modules[__name__].__dict__.clear()

In [ ]:
len_YearAF = 17
waterYearLen = 19
months_total = 228
path = '/Users/yunxiazhao/Documents/AAA_burnSnow/data8_10km_monthly/data_100km_3/'

def globalTOcontinent(dataTag):
    Data_Burn_2003 = np.load(path + dataTag + '_fireYear_2003.npy')
    Data_Burn_2004 = np.load(path + dataTag + '_fireYear_2004.npy')
    Data_Burn_2005 = np.load(path + dataTag + '_fireYear_2005.npy')
    basin_ID = [3020024310, 4020050210, 4020050220, 1020040190, 4020050290, 4020050470, 7020000010, 6020000010, 3020000010, 1020000010, 8020000010, 4020000010, 5020000010, 2020000010, 9020000010, 7020014250, 6020006540, 3020003790, 8020008900, 4020006940, 5020015660, 2020003440, 7020021430, 3020005240, 1020018110, 8020010700, 4020015090, 5020037270, 2020018240, 7020024600, 3020008670, 1020021940, 8020020760, 4020024190, 2020024230, 7020038340, 3020009320, 1020027430, 8020022890, 4020034510, 5020054880, 2020033490, 1020034170, 8020032840, 2020041390, 7020046750, 7020047840, 6020029280, 8020044560, 2020057170, 7020065090, 2020065840, 2020071190]
    continent_ID = [1, 2, 3, 4, 5, 6, 7, 8, 9]

    count_sum = 0
    data_all = np.zeros((3,waterYearLen, 12, 9,  len_var))

    for year in range(0, 3):
        if year == 0:
            Data_fireYear = Data_Burn_2003
        if year == 1:
            Data_fireYear = Data_Burn_2004
        if year == 2:
            Data_fireYear = Data_Burn_2005
 
            
            
        for i in range(0, 9):
            basin = Data_fireYear[:, :, :, 2]
            count = np.count_nonzero(np.floor(basin/1000000000) == continent_ID[i])
            index = np.where(np.floor(basin/1000000000) == continent_ID[i])

            index_x = index[0]
            index_y = index[1]
            index_z = index[2]
            rows = int(count / waterYearLen / 12)
            item_biomeType = np.zeros((waterYearLen, 12, rows, len_var))
            item_biomeType[:] = np.nan
            for j in range(0, count):
                item_biomeType[index_x[j], index_y[j], j % rows, :] = Data_fireYear[index_x[j], index_y[j], index_z[j], :]
            data_all[year, :, :, i, :] = np.nanmean(item_biomeType, axis=2)
    output = path + dataTag  + '_forEachContinent'
    np.save(output, data_all)

dataTag = 'Burn'
globalTOcontinent(dataTag)

dataTag = 'Unbn'
globalTOcontinent(dataTag)

dataTag = 'Dif'
globalTOcontinent(dataTag)


In [ ]:
import sys
sys.modules[__name__].__dict__.clear()